In [111]:
# 1. Импортируем библиотеки
import pandas as pd
import requests
from bs4 import BeautifulSoup
import category_encoders as ce

In [112]:
# 2. Считываем датафрейм.
path = ("_data.csv")
df = pd.read_csv(path)

In [113]:
# 3. Устнавливаем опцию отображения всех столбцов.
pd.set_option('display.max_columns', None)

In [114]:
# 4. Оставляем в датафрейме объявление только по Москве.
df = df[df["Адрес"].str.contains("Москва", case=False)]
len(df)

19737

In [115]:
# 5. Ищем дубликаты объявлений по описанию.
duplicate_rows = df[df.duplicated(subset='Описание')]
len(duplicate_rows)

281

In [116]:
# 6. Удаляем дубликаты объявлений по описанию.
df = df.drop_duplicates(subset='Описание')
len(df)

19456

In [117]:
# 7. Удаляем лишние столбцы, которые не повлияют или незначительно повлияют на результат.
df.drop(["Unnamed: 0", "Тип", "Телефоны", "Описание", "Площадь комнат, м2", "Дополнительно", "Название ЖК", "Серия дома", "Ссылка на объявление"], inplace=True, axis=1)

In [118]:
# 8. Меняем название столбцов.
rename_map = {'ID  объявления': 'ID', 'Количество комнат': "number_of_rooms", 'Метро': "metro", 'Адрес': "address", 'Площадь, м2': "apartment_square", 
              'Дом': "house", 'Парковка': "parking_lot", 'Можно с детьми/животными': 'children_pets_allowed','Высота потолков, м': 'ceiling_height',
              'Лифт': 'elevator','Мусоропровод': 'garbage_chute', 'Цена': 'price', 'Ремонт': 'decor', 'Балкон': 'balcony', 'Окна': 'windows', 
              'Санузел': 'bathroom'}

df.rename(columns=rename_map, inplace=True)

In [119]:
# 9. Заполняем пропущенные значения
df["number_of_rooms"].fillna("Апартаменты/свободная планировка", inplace=True)
df["metro"].fillna("Нет_данных", inplace=True)
df["parking_lot"].fillna("Отсутствует", inplace=True)

df["ceiling_height"] = df["ceiling_height"].fillna(df["ceiling_height"].mean())
df["elevator"] = df["elevator"].fillna("Нет данных")
df["garbage_chute"] = df["garbage_chute"].fillna("Нет_данных")
df["children_pets_allowed"] = df["children_pets_allowed"].fillna("Нет_данных")

df["decor"].fillna("Нет_данных", inplace=True)
df["balcony"].fillna("Отсутствует", inplace=True)
df["windows"].fillna("Нет_данных", inplace=True)
df["bathroom"].fillna("Нет_данных", inplace=True)

In [120]:
# 10. Напишем функцию определения количества лифтов в доме и ванных комнат в квартире.
def filter_values(lst, st):
    for i in lst:
        if st in i:
            for z in i:
                if z in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                    return int(z)

In [121]:
# 11. Обработаем категории children_pets_allowed, elevator, bathroom, windows, balcony, decor, parking_lot.
df["children_yes"] = df["children_pets_allowed"].apply(
    lambda x: 1 if "Можно с детьми" in x else 0
)
df["pets_yes"] = df["children_pets_allowed"].apply(
    lambda x: 1 if "Можно с животными" in x else 0
)
df["children_and_pets_yes"] = df["children_pets_allowed"].apply(
    lambda x: 1 if x == "Можно с детьми, Можно с животными" else 0
)
df["elevator_type"] = df["elevator"].apply(lambda x: len(x.split(",")))
df["elevator_pass"] = df["elevator"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Пасс") == None
    else filter_values(x.split(","), "Пасс")
)
df["elevator_cargo"] = df["elevator"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Груз") == None
    else filter_values(x.split(","), "Груз")
)
df["garbage_chute_ yes"] = df["garbage_chute"].apply(lambda x: 1 if x == "Да" else 0)
df["bathroom_type"] = df["bathroom"].apply(lambda x: len(x.split(",")))
df["bathroom_comb"] = df["bathroom"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Совмещенный") == None
    else filter_values(x.split(","), "Совмещенный")
)
df["bathroom_separate"] = df["bathroom"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Раздельный") == None
    else filter_values(x.split(","), "Раздельный")
)

df["bathroom_separate"] = df["bathroom"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Раздельный") == None
    else filter_values(x.split(","), "Раздельный")
)
df["windows_street"] = df["windows"].apply(lambda x: 1 if "На улицу" in x else 0)
df["windows_yard"] = df["windows"].apply(lambda x: 1 if "Во двор" in x else 0)
df["windows_all"] = df["windows"].apply(lambda x: 1 if "На улицу и двор" in x else 0)

df["balcony_type"] = df["balcony"].apply(
    lambda x: 0 if x == "Отсутствует" else len(x.split(","))
)
df["balcony_bal"] = df["balcony"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Балкон") == None
    else filter_values(x.split(","), "Балкон")
)
df["balcony_loggia"] = df["balcony"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Лоджия") == None
    else filter_values(x.split(","), "Лоджия")
)

df["decor_designer"] = df["decor"].apply(lambda x: 1 if x == "Дизайнерский" else 0)
df["decor_evro"] = df["decor"].apply(lambda x: 1 if x == "Евроремонт" else 0)
df["decor_cosmetic"] = df["decor"].apply(lambda x: 1 if x == "Косметический" else 0)

df["parking_lot_under"] = df["parking_lot"].apply(
    lambda x: 1 if x == "подземная" else 0
)
df["parking_lot_ground"] = df["parking_lot"].apply(
    lambda x: 1 if x == "наземная" else 0
)
df["parking_lot_open"] = df["parking_lot"].apply(lambda x: 1 if x == "открытая" else 0)
df["parking_multi-level"] = df["parking_lot"].apply(
    lambda x: 1 if x == "многоуровневая" else 0
)

In [122]:
# 12. Напишем функцию получения текущего курса валют.
def get_exchange_rate(currency):
    url = "https://www.cbr.ru/currency_base/daily/"  # Сохраняем URL веб-страницы Центробанка России, где содержится информация о ежедневных курсах валют
    response = requests.get(
        url
    )  # Отправляем HTTP-запрос на указанный URL, чтобы получить содержимое веб-страницы.
    soup = BeautifulSoup(
        response.content, "html.parser"
    )  # Используем библиотеку BeautifulSoup для парсинга (анализа) содержимого HTML веб-страницы.
    for row in soup.find_all(
        "tr"
    ):  # Перебираем все элементы tr (строки) в HTML-таблице на веб-странице.
        cells = row.find_all(
            "td"
        )  # Для каждой строки  находим все ячейки (элементы td) в этой строке.
        if (
            len(cells) > 1 and cells[1].text.strip() == currency
        ):  # Проверяем, что количество ячеек больше 1 (чтобы избежать ошибок) и текст второй ячейки соответствует переданному коду валюты.
            return float(
                cells[4].text.replace(",", ".")
            )  # Если условие выше выполнено, то мы возвращаем курс валюты, который находится в пятой ячейке таблицы (индекс 4), и заменяем запятую на точку для правильного чтения десятичной дроби в Python.

In [123]:
# 13. Напишем функцию конвертации валют.
def convert_price(x):
    try:
        if "руб" in x:
            return float(x.split()[0])
        elif "$" in x:
            return float(x.split()[0]) * get_exchange_rate("USD")
        else:
            return float(x.split()[0]) * get_exchange_rate("EUR")
    except:
        return 0

In [124]:
# 14. Заменяем значения на числовые значения в столбце price.
df["price"] = df["price"].apply(lambda x: convert_price(x))

In [125]:
# 15. Проставляем общую площадь в столбце apartment_square.
df["apartment_square"] = df["apartment_square"].apply(
    lambda x: x.split("/")[0] if isinstance(x, str) else x
)

In [126]:
# 16. Разделяем столбец house на 2 столбца: floor-nfloors, house_type.
df[["floor-nfloors", "house_type"]] = df["house"].str.split(", ", expand=True)

In [127]:
# 17. Разделяем столбец floor-nfloors на 2 столбца: floor, nfloors.
df[["floor", "nfloors"]] = df["floor-nfloors"].str.split("/", expand=True)

In [128]:
# 18. Проверяем какие значения в столбце house_type.
df["house_type"].unique()

array(['Монолитный', 'Монолитно-кирпичный', None, 'Панельный',
       'Сталинский', 'Кирпичный', 'старый фонд', 'Блочный', 'Деревянный',
       'Щитовой'], dtype=object)

In [129]:
# 19. Заменяем пустые значения.
df["house_type"].fillna("Нет_данных", inplace=True)

In [130]:
# 20. Для кодировки столбца house_type применяем метод Target Encoding,
# который подставит медианное значение стоимости квартиры для соответствующей категории столбца house_type.

# Инициализация Target Encoder.
target_encoder = ce.TargetEncoder(
    cols=["house_type"], min_samples_leaf=10, handle_unknown="value"
)

# Применение Target Encoder к датасету без явного указания цены.
df["house_type_fit_to_price"] = target_encoder.fit_transform(
    df["house_type"], df["price"]
)

In [131]:
# 21. Удаляем оставшиеся столбцы
df.drop(
    [
        "house",
        "floor-nfloors",
        "house_type",
        "decor",
        "balcony",
        "windows",
        "bathroom",
        "children_pets_allowed",
        "elevator",
        "garbage_chute",
        "parking_lot",
    ],
    inplace=True,
    axis=1,
)

In [132]:
# 22. Выводим финальный датасет в файл.
df.to_csv("data.csv")